In this notebook, unique entities that should be subjects are extracted from the data
Subjects to be extracted: 
- Countries (exported from pycountry)
- Companies
- Which companies own the mines 
- primary Commodities

In [39]:
import pandas as pd
import geopandas as gpd
import pycountry
import numpy as np

# import data
facilities = gpd.read_file("intermediate/data_merged.gpkg", layer = "facilities")
ownership = pd.read_csv("manual_input_data/ownership_cleaned.csv")

## Construct dataframe that links mine to company

In [40]:
# split strings by ","
ownership_df = ownership["owners"].str.split(",", expand=True)

# combine again with ids and years
ownership_df = pd.concat([ownership.loc[:,["facility_id", "year"]], ownership_df], axis = 1)

# pivot to tidy dataframe
ownership_df = pd.melt(ownership_df, id_vars=['facility_id', "year"], var_name='owner_nr', value_name='company')

# delete rows with NULL values
ownership_df = ownership_df[ownership_df.notnull().all(axis=1)]

# extract the ownership percentage and delete from old column
ownership_df['percentage'] = ownership_df['company'].str.extract(r'\((.*?)\)')
ownership_df['company'].replace(r'\((.*?)\)', "", inplace = True, regex = True)
ownership_df["company"] = ownership_df["company"].str.strip() # remove leading and trailing spaces

# add IRIs
ownership_df["company_iri"] = ownership_df["company"].replace(r'[^a-zA-Z\s]', '', regex = True) # remove all non-letters for iri
ownership_df["company_iri"] = ownership_df["company_iri"].str.strip() # remove leading and trailing spaces
ownership_df["company_iri"].replace(r'[\s]', '_', regex = True, inplace = True) #replace spaces with underscores in iri

In [41]:
ownership_df

,facility_id,year,owner_nr,company,percentage,company_iri
0,COM00001.00,2018,0,Hudbay Minerals Inc,100%,Hudbay_Minerals_Inc
1,COM00005.00,2019,0,Rusal Plc.,100%,Rusal_Plc
2,COM00006.00,2019,0,ArcelorMittal S.A.,100%,ArcelorMittal_SA
3,COM00007.00,2018,0,AngloGold Ashanti,100%,AngloGold_Ashanti
4,COM00009.00,2017,0,Newmont Mining Corporation,100%,Newmont_Mining_Corporation
...,...,...,...,...,...,...
2277,COM00397.00,2018,3,JX Nippon Mining and Metals,10%,JX_Nippon_Mining_and_Metals
2496,COM00840.00,2019,3,SMM Morenci Inc.,13%,SMM_Morenci_Inc
2523,COM00885.00,2020,3,Daewoo International Corporation and Korea Re...,7.5%,Daewoo_International_Corporation_and__Korea_Re...
2579,COM01010.00,2019,3,POS-Ore,20%,POSOre


In [42]:
# get only the most recent owners 
idx = ownership_df.groupby("facility_id")["year"].transform(max) == ownership_df["year"]
print(len(ownership_df))

ownership_df = ownership_df[idx]
len(ownership_df)

819


788

In [43]:
# Tests:
# this mine has ownership values for multiple years, so only the most recent one should be displayed (only one row)
assert len(ownership_df[ownership_df["facility_id"] == "COM00400.00"]) == 1

# this mine has multiple ownership values for one year, so all of them should be displayed
assert len(ownership_df[ownership_df["facility_id"] == "COM00048.00"]) > 1

display(ownership_df[ownership_df["facility_id"] == "COM00400.00"])
display(ownership_df[ownership_df["facility_id"] == "COM00048.00"])

,facility_id,year,owner_nr,company,percentage,company_iri
205,COM00400.00,2019,0,Rusal Plc.,100%,Rusal_Plc


,facility_id,year,owner_nr,company,percentage,company_iri
17,COM00048.00,2018,0,Teck Resources Limited,22.5%,Teck_Resources_Limited
712,COM00048.00,2018,1,BHP Billiton plc,33.75%,BHP_Billiton_plc
1407,COM00048.00,2018,2,Glencore plc,33.75%,Glencore_plc
2102,COM00048.00,2018,3,Mitsubishi Corporation,10%,Mitsubishi_Corporation


## Construct dataframe with unique companies

In [44]:
companies_df = ownership_df.loc[:,["company", "company_iri"]].drop_duplicates()

## Construct dataframe with all countries

In [45]:
country_alpha_3 = []
country_name = []
for country in list(pycountry.countries):
    country_alpha_3.append(country.alpha_3)
    country_name.append(country.name)

countries_df = pd.DataFrame({"country_alpha_3":  country_alpha_3, "country_name":  country_name})

## Extract unique GADM regions

In [46]:
facilities

,facility_id,facility_name,facility_other_names,sub_site_name,sub_site_other_names,facility_type,commodities_products,facility_equipment,production_start,production_end,...,GID_0,GID_1,GID_2,GID_3,GID_4,source_id,comment,AREA,primary_commodity,geometry
0,COM00001.00,777,NaN,NaN,NaN,Mine,"Copper, Zinc, Gold, Silver",Underground,NaN,NaN,...,CAN,CAN.3_1,CAN.3.13_1,CAN.3.13.2_1,NaN,det_1149,NaN,6.342012,Me.Cu,MULTIPOINT (-101.87946 54.77482)
1,COM00002.00,A Narrain,NaN,NaN,NaN,Mine,Iron ore,Open pit,NaN,NaN,...,IND,IND.16_1,IND.16.11_1,IND.16.11.2_1,NaN,det_1307,NaN,2.107415,Me.Fe,MULTIPOINT (76.20841 14.22353)
2,COM00003.00,Absaloka,NaN,NaN,NaN,Mine,Sub-bituminous coal,NaN,1974.0,NaN,...,USA,USA.27_1,USA.27.2_1,NaN,NaN,det_1427,NaN,0.522208,F.coal,MULTIPOINT (-107.08290 45.80531)
3,COM00007.00,AGA Mineracao,NaN,NaN,NaN,Mine,Gold,"Underground, Open pit, Heap leaching plant",NaN,NaN,...,BRA,BRA.13_1,BRA.13.662_1 ; BRA.13.672_1,BRA.13.662.2_1 ; BRA.13.662.4_1 ; BRA.13.672.1...,NaN,det_1057,NaN,1.249552,Me.Au,"MULTIPOINT (-43.73907 -19.86773, -43.76980 -19..."
4,COM00008.00,Aguablanca,NaN,NaN,NaN,Mine,"Nickel, Copper, Cobalt, Gold, PGM",Underground,2005.0,2016.0,...,ESP,ESP.11_1,ESP.11.1_1,ESP.11.1.10_1,ESP.11.1.10.11_1,det_1205,NaN,3.191783,Me.Ni,MULTIPOINT (-6.18033 37.96558)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088,COM01428.00,Zhunge’er Mines,Shenhua Zhunge’er Energy Co.,NaN,NaN,Mine,"Coal, Long flame coal",Open pit,NaN,NaN,...,CHN,CHN.19_1,CHN.19.7_1,CHN.19.7.5_1,NaN,det_1376,NaN,63.669780,F.coal,"MULTIPOINT (111.25019 39.79218, 111.25700 39.7..."
1089,COM01429.00,Zibulo,Zondagsfontein,NaN,NaN,Mine,Thermal coal,NaN,NaN,NaN,...,ZAF,ZAF.6_1,ZAF.6.3_1,ZAF.6.3.3_1,ZAF.6.3.3.24_1,det_1103,NaN,0.512426,F.coal,MULTIPOINT (29.01740 -26.20006)
1090,COM01430.00,Zijinshan,Fujian Zijinshan,NaN,NaN,Mine,"Gold, Copper, Copper cathodes, Silver",Open Pit,1980.0,NaN,...,CHN,CHN.4_1,CHN.4.2_1,CHN.4.2.4_1,NaN,det_1315,NaN,18.908502,Me.Au,MULTIPOINT (116.40544 25.18979)
1091,COM01432.00,Zinkgruvan,NaN,NaN,NaN,Mine,"Lead, Silver, Copper, Zinc",NaN,1857.0,NaN,...,SWE,SWE.11_1,SWE.11.1_1,NaN,NaN,det_1075,NaN,NaN,Me.Zn,MULTIPOINT (15.10346 58.81644)


In [47]:
# Extract all GADM regions that are present in the mining data
gid = facilities.loc[:, facilities.columns.str.startswith(("facility_id",'GID'))]
display(gid)
print(len(gid))

,facility_id,GID_0,GID_1,GID_2,GID_3,GID_4
0,COM00001.00,CAN,CAN.3_1,CAN.3.13_1,CAN.3.13.2_1,NaN
1,COM00002.00,IND,IND.16_1,IND.16.11_1,IND.16.11.2_1,NaN
2,COM00003.00,USA,USA.27_1,USA.27.2_1,NaN,NaN
3,COM00007.00,BRA,BRA.13_1,BRA.13.662_1 ; BRA.13.672_1,BRA.13.662.2_1 ; BRA.13.662.4_1 ; BRA.13.672.1...,NaN
4,COM00008.00,ESP,ESP.11_1,ESP.11.1_1,ESP.11.1.10_1,ESP.11.1.10.11_1
...,...,...,...,...,...,...
1088,COM01428.00,CHN,CHN.19_1,CHN.19.7_1,CHN.19.7.5_1,NaN
1089,COM01429.00,ZAF,ZAF.6_1,ZAF.6.3_1,ZAF.6.3.3_1,ZAF.6.3.3.24_1
1090,COM01430.00,CHN,CHN.4_1,CHN.4.2_1,CHN.4.2.4_1,NaN
1091,COM01432.00,SWE,SWE.11_1,SWE.11.1_1,NaN,NaN


1093


In [48]:
# split the ";"-separated values and explode the resulting lists for all columns
# do this to every row
for row in range(len(gid)):
    facility = gid.iloc[row,0]

    # select row
    temp_df = gid.drop("facility_id", axis=1).iloc[row,:].dropna()
    max_gadm_level = len(temp_df)-1
    rightmost = temp_df[-1]

    # split it by ";"
    gadm_list = rightmost.split(';')
    gadm_list = [x.strip(' ') for x in gadm_list] # clean whitepsace

    # construct cols for dataframe to append
    fac_id_list = [facility] * len(gadm_list)

    # construct output df
    output_df = pd.DataFrame(list(zip(fac_id_list, gadm_list)), columns = ["facility_id", "GID"])

    if row == 0:
        facilities_gadm = output_df

    else: 
        facilities_gadm = pd.concat([facilities_gadm, output_df]).reset_index(drop=True)

facilities_gadm

,facility_id,GID
0,COM00001.00,CAN.3.13.2_1
1,COM00002.00,IND.16.11.2_1
2,COM00003.00,USA.27.2_1
3,COM00007.00,BRA.13.662.2_1
4,COM00007.00,BRA.13.662.4_1
...,...,...
1221,COM01428.00,CHN.19.7.5_1
1222,COM01429.00,ZAF.6.3.3.24_1
1223,COM01430.00,CHN.4.2.4_1
1224,COM01432.00,SWE.11.1_1


## Export data

In [49]:
import os

# create the intermediate directory if it does not exist
path = "./intermediate"
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

pd.DataFrame(companies_df).to_csv("intermediate/companies.csv", index = False)
pd.DataFrame(countries_df).to_csv("intermediate/countries.csv", index = False)
pd.DataFrame(ownership_df).to_csv("intermediate/ownership.csv", index = False)
pd.DataFrame(facilities_gadm).to_csv("intermediate/gadm.csv", index = False)
